In [1]:
import librosa
import os
import soundfile as sf
import numpy as np
import math
import pickle

In [2]:
sr = 44100
audio_vectors = {}
wav_file_path = '/home/rashmi/Dataset/pre-processed/single_file/test_wav/ses05m_impro06_M004.wav'

orig_wav_vector, _sr = librosa.load(wav_file_path, sr=sr)
orig_wav_vector, _sr

start_time = 0
end_time = 13
start_frame = math.floor(start_time * sr)
end_frame =  math.floor(end_time * sr)

print (orig_wav_vector)
print (start_frame,end_frame)

truncated_wav_vector = orig_wav_vector[start_frame:end_frame + 1]
print (truncated_wav_vector)

truncated_wav_file_name= "Ses05F_impro08"
audio_vectors[truncated_wav_file_name] = truncated_wav_vector

with open('/home/rashmi/Dataset/pre-processed/single_file/audio_vectors.pkl', 'wb') as f:
        pickle.dump(audio_vectors, f)

[0. 0. 0. ... 0. 0. 0.]
0 573300
[0.         0.         0.         ... 0.00021359 0.00019978 0.00022871]


In [3]:
import os
import pickle
import soundfile as sf
import numpy as np
from tqdm import tqdm
import librosa
import math
import random
import pandas as pd

In [4]:
audio_vectors_path = '/home/rashmi/Dataset/pre-processed/single_file/audio_vectors.pkl'
audio_vectors = pickle.load(open(audio_vectors_path, 'rb'))
random_file_name = list(audio_vectors.keys())[random.choice(range(len(audio_vectors.keys())))]
y = audio_vectors[random_file_name]
sr = 44100

feature_list = [random_file_name];
sig_mean = np.mean(abs(y))
feature_list.append(sig_mean)  # sig_mean
feature_list.append(np.std(y))  # sig_std

rms = librosa.feature.rms(y + 0.0001)[0]
feature_list.append(np.mean(rms))  # rms_mean
feature_list.append(np.std(rms))  # rms_std

silence = 0
for e in rms:
    if e <= 0.4 * np.mean(rms):
        silence += 1
silence /= float(len(rms))
feature_list.append(silence)  # silence

y_harmonic = librosa.effects.hpss(y)[0]
feature_list.append(np.mean(y_harmonic) * 1000)  # harmonic (scaled by 1000)

# based on the pitch detection algorithm mentioned here:
# http://access.feld.cvut.cz/view.php?cisloclanku=2009060001
cl = 0.45 * sig_mean
center_clipped = []
for s in y:
    if s >= cl:
        center_clipped.append(s - cl)
    elif s <= -cl:
        center_clipped.append(s + cl)
    elif np.abs(s) < cl:
        center_clipped.append(0)
auto_corrs = librosa.core.autocorrelate(np.array(center_clipped))
feature_list.append(1000 * np.max(auto_corrs)/len(auto_corrs))  # auto_corr_max (scaled by 1000)
feature_list.append(np.std(auto_corrs))  # auto_corr_std

columns = ['wav_file', 'sig_mean', 'sig_std', 'rmse_mean', 'rmse_std', 'silence', 'harmonic', 'auto_corr_max', 'auto_corr_std']
df_features = pd.DataFrame(columns=columns)

df_features = df_features.append(pd.DataFrame(feature_list, index=columns).transpose(), ignore_index=True)

df_features.to_csv('/home/rashmi/Dataset/pre-processed/single_file/audio_features.csv', index=False)

In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
import matplotlib
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.class_weight import compute_class_weight
import itertools
from IPython.display import display

%matplotlib inline

In [6]:
x_train = pd.read_csv('/home/rashmi/Dataset/pre-processed/s2e/audio_train.csv')
#x_test = pd.read_csv('/home/rashmi/Dataset/pre-processed/s2e/audio_test.csv')

print(x_train.shape)
y_train = x_train['label']
#y_test = x_test['label']

(1704, 10)


In [8]:
print(x_train.shape)#, x_test.shape)
cl_weight = dict(pd.Series(x_train['label']).value_counts(normalize=True))
print(dict(pd.Series(x_train['label']).value_counts()))

del x_train['label']
#del x_test['label']
del x_train['wav_file']
#del x_test['wav_file']

(1704, 10)
{2: 508, 1: 357, 5: 297, 3: 242, 4: 165, 0: 135}


In [9]:
x_test_new=pd.read_csv('/home/rashmi/Dataset/pre-processed/single_file/audio_features.csv')
del x_test_new['wav_file']
print (x_test_new)

   sig_mean   sig_std  rmse_mean  rmse_std   silence  harmonic  auto_corr_max  \
0  0.003807  0.006724   0.004809    0.0047  0.465179 -0.008355       0.034514   

   auto_corr_std  
0       0.189598  


In [10]:
emotion_dict = {'ang': 0,
                'hap': 1,
                'sad': 2,
                'fea': 3,
                'sur': 4,
                'neu': 5}

emo_keys = list(['ang', 'hap', 'sad', 'fea', 'sur', 'neu'])

In [11]:
rf_classifier = RandomForestClassifier(n_estimators=1200, min_samples_split=25)
xgb_classifier = xgb.XGBClassifier(max_depth=7, learning_rate=0.008, objective='multi:softprob', 
                                   n_estimators=1200, sub_sample=0.8, num_class=len(emotion_dict),
                                   booster='gbtree', n_jobs=4)
mnb_classifier = MultinomialNB()

In [12]:
eclf1 = VotingClassifier(estimators=[('rf', rf_classifier), ('xgb', xgb_classifier), ('mnb', mnb_classifier)], voting='soft')
eclf1 = eclf1.fit(x_train, y_train)
pred_probs=eclf1.predict_proba(x_test_new)
print (pred_probs)

#np.savetxt('/home/rashmi/Dataset/pre-processed/emo_val/rf.csv',pred_probs,delimiter=',')

[[0.06853257 0.26746664 0.37977596 0.06861857 0.0723886  0.14321766]]


In [13]:
index = pred_probs.argmax(axis=1).item()
index
emo_keys[index]

2

'sad'